#  STEP 6: Phylogenetics R&D (Phase 8)

**Goal:** To convert our final VCF into an alignment, and then use that alignment to build our final, publication-ready Phylogenetic Tree.

**Why (The Final Goal):**
This is the scientific climax of the project.
1.  **Conversion (The "Recipe"):** Our "Golden File" (`ANALYSIS_READY.vcf.gz`) is a *list of differences*. Tree-builders like `IQ-TREE` need an *alignment* (a FASTA or PHYLIP file). We will test our new tool (`snp-sites`) to perform this conversion.
2.  **Tree Building (The "R&D"):** We will test our primary analysis tool (`iqtree`) on this new alignment to generate the final tree.

##  1: Handoff from "The Factory" (Phase 7 Success)

**The Handoff (Done):**
We successfully updated our `Snakefile` (V6.0) to automate the `merge` and `filter` "recipes" developed in `Notebook 06`. We then executed this final factory step from the terminal:

```bash
snakemake --cores 1 --rerun-triggers mtime